In [1]:
import codecs
import pandas as pd
from pyvi import ViTokenizer
from underthesea import word_tokenize

df_vnexpress = pd.read_json(codecs.open('data/vnexpress.json','r','utf-8'))
df_dantri = pd.read_json(codecs.open('data/dantri.json','r','utf-8'))
df_vietnamnet = pd.read_json(codecs.open('data/vietnamnet.json','r','utf-8'))

data_vnexpress = pd.DataFrame(df_vnexpress.response['docs'])[['url','content']]
data_dantri = pd.DataFrame(df_dantri.response['docs'])[['url','content']]
data_vietnamnet = pd.DataFrame(df_vietnamnet.response['docs'])[['url','content']]

labels = [
('Chinh tri Xa hoi', 0),
('Cong Nghe', 1),
('Doi Song', 2),
('Du Lich', 3),
('Giai Tri', 4),
('Giao Duc', 5),
('Khoa hoc', 6),
('Kinh doanh', 7),
('Phap Luat ', 8),
('Suc Khoe', 9),
('The Gioi', 10),
('The Thao', 11),
('Xe', 12),
]

In [2]:
regex_list_filter_url_dantri= [
('^.*dantri\.com\.vn\/bong-da-tbn\/'),
('^.*dantri\.com\.vn\/bong-da-trong-nuoc\/'),
('^.*dantri\.com\.vn\/chinh-tri\/'),
('^.*dantri\.com\.vn\/dien-thoai\/'),
('^.*dantri\.com\.vn\/doi-song\/'),
('^.*dantri\.com\.vn\/du-lich\/'),
('^.*dantri\.com\.vn\/du-lich-kham-pha\/'),
('^.*dantri\.com\.vn\/giai-tri\/'),
('^.*dantri\.com\.vn\/giao-duc-khuyen-hoc\/'),
('^.*dantri\.com\.vn\/khoa-hoc\/'),
('^.*dantri\.com\.vn\/khoa-hoc-cong-nghe\/'),
('^.*dantri\.com\.vn\/khoa-hoc-doi-song\/'),
('^.*dantri\.com\.vn\/khuyen-hoc\/'),
('^.*dantri\.com\.vn\/kinh-doanh\/'),
('^.*dantri\.com\.vn\/lam-dep\/'),
('^.*dantri\.com\.vn\/o-to-xe-may\/'),
('^.*dantri\.com\.vn\/phap-luat\/'),
('^.*dantri\.com\.vn\/suc-khoe\/'),
('^.*dantri\.com\.vn\/suc-manh-so\/'),
('^.*dantri\.com\.vn\/the-gioi\/'),
('^.*dantri\.com\.vn\/the-thao\/'),
('^.*dantri\.com\.vn\/the-thao-quoc-te\/'),
('^.*dantri\.com\.vn\/the-thao-trong-nuoc\/'),
('^.*dantri\.com\.vn\/vi-tinh\/'),
('^.*dantri\.com\.vn\/xa-hoi\/'),
('^.*dantri\.com\.vn\/van-hoa\/'),    
]
regex_content_match_dantri = '^(.*?[0-9]{2}\/[0-9]{2}\/[0-9]{4} \- [0-9]{2}\:[0-9]{2})'

In [3]:
regex_list_filter_url_vnexpress= [
'^.*/vnexpress\.net\/bong-da\/',
'^.*/vnexpress\.net\/doi-song\/',
'^.*/vnexpress\.net\/du-lich\/',
'^.*/vnexpress\.net\/giai-tri\/',
'^.*/vnexpress\.net\/giao-duc\/',
'^.*/vnexpress\.net\/khoa-hoc\/',
'^.*/vnexpress\.net\/kinh-doanh\/',
'^.*/vnexpress\.net\/oto-xe-may\/',
'^.*/vnexpress\.net\/phap-luat\/',
'^.*/vnexpress\.net\/so-hoa\/',
'^.*/vnexpress\.net\/suc-khoe\/',
'^.*/vnexpress\.net\/the-gioi\/',
'^.*/vnexpress\.net\/the-thao\/',
]
regex_content_match_vnexpress = '^(.*?[0-9]{2}\:[0-9]{2} \(GMT\+7\))'

In [4]:
regex_list_filter_url_vietnamnet= [
'^.*vietnamnet\.vn\/vn\/cong-nghe\/',
'^.*vietnamnet\.vn\/vn\/doi-song\/',
'^.*vietnamnet\.vn\/vn\/giai-tri\/',
'^.*vietnamnet\.vn\/vn\/giao-duc\/',
'^.*vietnamnet\.vn\/vn\/kinh-doanh\/',
'^.*vietnamnet\.vn\/vn\/oto-xe-may\/',
'^.*vietnamnet\.vn\/vn\/phap-luat\/',
'^.*vietnamnet\.vn\/vn\/suc-khoe\/',
'^.*vietnamnet\.vn\/vn\/the-gioi\/',
'^.*vietnamnet\.vn\/vn\/the-thao\/',
]
regex_content_match_vietnamnet= '^(.*?[0-9]{2}\:[0-9]{2} GMT\+7)'

In [5]:
print('dantri: %d' %  data_dantri.shape[0])
print('express: %d' % data_vnexpress.shape[0])
print('vietnamnet: %d' % data_vietnamnet.shape[0])

dantri: 49071
express: 42314
vietnamnet: 9054


In [6]:
# Get all urls in domain https://dantri.com.vn
data_dantri = data_dantri[data_dantri.url.str.match(pat='^https://dantri\.com\.vn')]
# Get all urls in domain https://vnexpress.net
data_vnexpress = data_vnexpress[data_vnexpress.url.str.match(pat='^https://vnexpress\.net')]
# Get all urls in domain https://vietnamnet.vn
data_vietnamnet = data_vietnamnet[data_vietnamnet.url.str.match(pat='^https://(m\.)?vietnamnet\.vn')]

In [7]:
def filter_url(data, regex_list_filter_url, regex_content_match):
    data_filterd_url = pd.DataFrame()
    for regex in regex_list_filter_url:
        print( '%.5d:  ' % data[data.url.str.match(regex)].shape[0] +regex)
        data_filterd_url = data_filterd_url.append(data[data.url.str.match(regex)],ignore_index=True)

    data = data_filterd_url[data_filterd_url.content.str.match(regex_content_match)]
    print(data.shape)
    return data

In [8]:
def filter_short_data(data, num_dot):
    return data[data.content.str.count('[^\.]\.')>num_dot]

In [9]:
data_vnexpress = filter_url(data_vnexpress, regex_list_filter_url_vnexpress,regex_content_match_vnexpress)

data_dantri = filter_url(data_dantri,regex_list_filter_url_dantri,regex_content_match_dantri)

data_vietnamnet = filter_url(data_vietnamnet,regex_list_filter_url_vietnamnet,regex_content_match_vietnamnet)

02092:  ^.*/vnexpress\.net\/bong-da\/
01735:  ^.*/vnexpress\.net\/doi-song\/
02846:  ^.*/vnexpress\.net\/du-lich\/
04000:  ^.*/vnexpress\.net\/giai-tri\/
02495:  ^.*/vnexpress\.net\/giao-duc\/
01678:  ^.*/vnexpress\.net\/khoa-hoc\/
04017:  ^.*/vnexpress\.net\/kinh-doanh\/
01465:  ^.*/vnexpress\.net\/oto-xe-may\/
03060:  ^.*/vnexpress\.net\/phap-luat\/
02893:  ^.*/vnexpress\.net\/so-hoa\/
02626:  ^.*/vnexpress\.net\/suc-khoe\/
04061:  ^.*/vnexpress\.net\/the-gioi\/
01603:  ^.*/vnexpress\.net\/the-thao\/
(34532, 2)
00002:  ^.*dantri\.com\.vn\/bong-da-tbn\/
00015:  ^.*dantri\.com\.vn\/bong-da-trong-nuoc\/
00920:  ^.*dantri\.com\.vn\/chinh-tri\/
00002:  ^.*dantri\.com\.vn\/dien-thoai\/
01241:  ^.*dantri\.com\.vn\/doi-song\/
01194:  ^.*dantri\.com\.vn\/du-lich\/
00004:  ^.*dantri\.com\.vn\/du-lich-kham-pha\/
00568:  ^.*dantri\.com\.vn\/giai-tri\/
04246:  ^.*dantri\.com\.vn\/giao-duc-khuyen-hoc\/
00017:  ^.*dantri\.com\.vn\/khoa-hoc\/
01296:  ^.*dantri\.com\.vn\/khoa-hoc-cong-nghe\/
00007:  

In [10]:
regex_list_parse_vnexpress =[
 '^(.*?[0-9]{2}\:[0-9]{2} \(GMT\+7\))',
'(Quảng cáo Xem nhiều nhất.*)',
'(Tin liên quan\:.*)$',
'(Quảng cáo   Ý kiến bạn đọc.*)$',
'(Xem nhiều nhất.*)$',
'(Quảng cáo Quảng cáo.*)$',
'(Ý kiến bạn đọc.*)$',
'(Tags.*)$',
'(Quảng cáo  )$',
]

In [11]:
regex_list_parse_dantri=[
    '^(.*?[0-9]{2}\/[0-9]{2}/[0-9]{4} - [0-9]{2}\:[0-9]{2})',
    '^(.*?Chia sẻ Dân trí)',
    '(Tag : .*)$',
]

In [12]:
regex_list_parse_vietnamnet=[
 '^(.*?[0-9]{2}\:[0-9]{2} GMT\+7)',
    '(Gửi bình luận Chủ đề.*)$',
]

In [13]:
for regex in regex_list_parse_vnexpress:
    data_vnexpress = data_vnexpress.replace(to_replace=regex,regex=True,value='')

In [14]:
for regex in regex_list_parse_dantri:
    data_dantri = data_dantri.replace(to_replace=regex,regex=True,value='')

In [15]:
for regex in regex_list_parse_vietnamnet:
    data_vietnamnet = data_vietnamnet.replace(to_replace=regex,regex=True,value='')

In [16]:
data_vnexpress = filter_short_data(data_vnexpress,15)
data_dantri = filter_short_data(data_dantri,15)
data_vietnamnet = filter_short_data(data_vietnamnet,15)

In [17]:
maps = [
    ( '\/bong-da\/',11),
    ('\/doi-song\/',2),
    ('\/du-lich\/', 3),
    ('\/giai-tri\/', 4),
    ('\/giao-duc\/', 5),
    ('\/khoa-hoc\/', 6),
    ('\/kinh-doanh\/', 7),
    ('\/oto-xe-may\/', 12),
    ('\/phap-luat\/', 8),
    ('\/so-hoa\/', 1),
    ('\/suc-khoe\/',9),
    ('\/the-gioi\/',10),
    ('\/the-thao\/', 11),
]
count = [0 for x in range(13)]

for name,idx in maps:
    count[idx]+=data_vnexpress[data_vnexpress.url.str.contains(name)].shape[0]
    data_vnexpress.loc[data_vnexpress.url.str.contains(name), 'label'] = idx
for label, idx in labels:
    print('%s: %d' % (label,count[idx]))
print('Sum: %d' % sum(count))

Chinh tri Xa hoi: 0
Cong Nghe: 1725
Doi Song: 1329
Du Lich: 2173
Giai Tri: 3052
Giao Duc: 1517
Khoa hoc: 543
Kinh doanh: 3069
Phap Luat : 1460
Suc Khoe: 1783
The Gioi: 2048
The Thao: 2840
Xe: 744
Sum: 22283


In [18]:
maps = [
    ('\/cong-nghe\/', 1),
    ('\/doi-song\/', 2),
    ('\/doi-song\/du-lich\/', 3),
    ('\/giai-tri\/', 4),
    ('\/giao-duc\/', 5),
    ('\/kinh-doanh\/', 7),
    ('\/oto-xe-may\/', 12),
    ('\/phap-luat\/', 8),
    ('\/suc-khoe\/', 9),
    ('\/the-gioi\/', 10),
    ('\/the-thao\/', 11),
]
count = [0 for x in range(13)]

for name,idx in maps:
    count[idx]+=data_vietnamnet[data_vietnamnet.url.str.contains(name)].shape[0]
    data_vietnamnet.loc[data_vietnamnet.url.str.contains(name), 'label'] = idx
for label, idx in labels:
    print('%s: %d' % (label,count[idx]))
print('Sum: %d' % sum(count))

Chinh tri Xa hoi: 0
Cong Nghe: 102
Doi Song: 179
Du Lich: 19
Giai Tri: 178
Giao Duc: 475
Khoa hoc: 0
Kinh doanh: 474
Phap Luat : 126
Suc Khoe: 61
The Gioi: 191
The Thao: 284
Xe: 484
Sum: 2573


In [19]:
maps = [
    ('\/bong-da-tbn\/', 11),
    ('\/bong-da-trong-nuoc\/', 11),
    ('\/chinh-tri\/', 0),
    ('\/dien-thoai\/', 1),
    ('\/doi-song\/', 2),
    ('\/du-lich\/', 3),
    ('\/du-lich-kham-pha\/', 3),
    ('\/giai-tri\/', 4),
    ('\/giao-duc-khuyen-hoc\/', 5),
    ('\/khoa-hoc\/', 6),
    ('\/khoa-hoc-cong-nghe\/', 6),
    ('\/khoa-hoc-doi-song\/', 6),
    ('\/khuyen-hoc\/', 5),
    ('\/kinh-doanh\/', 7),
    ('\/lam-dep\/', 9),
    ('\/o-to-xe-may\/', 12),
    ('\/phap-luat\/', 8),
    ('\/suc-khoe\/', 9),
    ('\/suc-manh-so\/', 1),
    ('\/the-gioi\/', 10),
    ('\/the-thao\/', 11),
    ('\/the-thao-quoc-te\/', 11),
    ('\/the-thao-trong-nuoc\/', 11),
    ('\/van-hoa\/', 4),
    ('\/vi-tinh\/', 1),
    ('\/xa-hoi\/', 0),
]
count = [0 for x in range(13)]

for name,idx in maps:
    count[idx]+=data_dantri[data_dantri.url.str.contains(name)].shape[0]
    data_dantri.loc[data_dantri.url.str.contains(name), 'label'] = idx
for label, idx in labels:
    print('%s: %d' % (label,count[idx]))
    
print('Sum: %d' % sum(count))

Chinh tri Xa hoi: 3239
Cong Nghe: 743
Doi Song: 980
Du Lich: 762
Giai Tri: 1036
Giao Duc: 3395
Khoa hoc: 606
Kinh doanh: 3651
Phap Luat : 1524
Suc Khoe: 873
The Gioi: 4025
The Thao: 3405
Xe: 284
Sum: 24523


In [20]:
print('dantri: %d' %  data_dantri.shape[0])
print('express: %d' % data_vnexpress.shape[0])
print('vietnamnet: %d' % data_vietnamnet.shape[0])

dantri: 24523
express: 22283
vietnamnet: 2554


In [21]:
data_vietnamnet['content'] = data_vietnamnet.apply(lambda row: word_tokenize(row['content'], format="text"), axis=1)

data_dantri['content'] = data_dantri.apply(lambda row: word_tokenize(row['content'], format="text"), axis=1)

data_vnexpress['content'] = data_vnexpress.apply(lambda row: word_tokenize(row['content'], format="text"), axis=1)

In [22]:
data_dantri.url.to_csv('url.csv',header=False,index=False)
data_vnexpress.url.to_csv('url2.csv',header=False,index=False)
data_vietnamnet.url.to_csv('url3.csv',header=False,index=False)

In [23]:
data_dantri.to_csv('filtered_data/dantri.csv',index=False, header=True)
data_vnexpress.to_csv('filtered_data/vnexpress.csv',index=False,header=True)
data_vietnamnet.to_csv('filtered_data/vietnamnet.csv',index=False,header=True)

In [24]:
f = open('vietnamese-stopwords-dash.txt', 'r')
x = f.read().splitlines()
f.close()


In [25]:
type(x)

list

In [26]:
x

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [27]:
sentence ="C.Ronaldo chấn thương, Bồ Đào Nha hòa thất vọng Serbia Chia sẻ Dân trí  Bồ Đào Nha tiếp tục gây thất vọng khi tiếp tục để Serbia cầm hòa 1-1 trên sân nhà ở bảng B vòng loại Euro 2020. C.Ronaldo dính chấn thương rời sân ở phút 30, Danilo là người ghi bàn cho nhà đương kim vô địch châu Âu. >>Tuyển Anh lội ngược dòng thắng đậm trên sân của Montenegro >>Hàng công thăng hoa, Pháp đại thắng \"4 sao\" trước Iceland C.Ronaldo chấn thương, Bồ Đào Nha hòa thất vọng Serbia Sau trận hòa Ukraine 0-0 ở lượt đấu trước, Bồ Đào Nha chịu khá nhiều áp lực. Một chiến thắng trước Serbia là điều mà các nhà vô địch châu Âu cần để giải tỏa áp lực, đồng thời khẳng định vị thế tại bảng B vòng loại Euro 2020. Tuy nhiên, mọi thứ diễn ra không đúng với tính toán của Bồ Đào Nha. Chỉ sau những sơ suất không đáng có của hàng thủ, Bồ Đào Nha đã chịu quả penalty và Tadic đưa Serbia dẫn trước từ khá sớm. C.Ronaldo gặp chấn thương đen đủi và phải rời sân ở phút 30. Bồ Đào Nha vẫn chơi ép sân, liên tục tạo ra các cơ hội và nỗ lực đó đã được đền đáp với cú “nã đại bác” thành bàn của Danilo. Dù rất nỗ lực, tuy nhiên Bồ Đào Nha không thể ghi bàn và chấp nhận kết quả hòa 1-1 chung cuộc. Đây là trận hòa thứ hai liên tiếp của Bồ Đào Nha trên sân nhà và họ tụt xuống vị trí thứ ba cở bảng B, đứng sau cả Ukraine lẫn Luxemburg. Với C.Ronaldo, mọi thứ càng trở nên tồi tệ hơn. Với chấn thương ở bắp chân, anh nhiều khả năng sẽ nghỉ thi đấu hai tuần và chắc chắn vắng mặt khi Juventus đối đầu Ajax ở tứ kết lượt đi Champions League. Bồ Đào Nha tung ra sân đội hình thiên về tấn công với Danilo, William Carvalho, Rafa Silva ở giữa sân còn Bernardo, Dyego Sousa sát cánh cùng C.Ronaldo. Những phút đầu tiên diễn ra với tốc độ cao, cả hai đội đều muốn có bàn thắng sớm. Đội hình thi đấu Bồ Đào Nha "

In [28]:
word_tokenize(sentence, format="text")

'C. Ronaldo chấn_thương , Bồ_Đào_Nha_hòa thất_vọng Serbia Chia_sẻ Dân_trí Bồ_Đào_Nha tiếp_tục gây thất_vọng khi tiếp_tục để Serbia cầm hòa_1-1 trên sân nhà ở bảng B vòng loại Euro 2020 . C._Ronaldo dính chấn_thương rời sân ở phút 30 , Danilo là người ghi_bàn cho nhà đương_kim vô_địch châu_Âu . >> Tuyển_Anh lội ngược dòng thắng đậm trên sân của Montenegro >> Hàng công thăng_hoa , Pháp đại_thắng " 4 sao " trước Iceland_C._Ronaldo chấn_thương , Bồ_Đào_Nha_hòa thất_vọng Serbia Sau trận hòa_Ukraine 0-0 ở lượt đấu trước , Bồ_Đào Nha chịu khá nhiều áp_lực . Một chiến_thắng trước Serbia là điều mà các nhà vô_địch châu_Âu cần để giải_tỏa áp_lực , đồng_thời khẳng_định vị_thế tại bảng B vòng loại Euro 2020 . Tuy_nhiên , mọi thứ diễn ra không đúng với tính_toán của Bồ_Đào Nha ._Chỉ sau những sơ_suất không đáng có của hàng thủ , Bồ_Đào Nha đã chịu quả penalty và Tadic đưa Serbia dẫn trước từ khá sớm ._C._Ronaldo gặp chấn_thương đen_đủi và phải rời sân ở phút 30 . Bồ_Đào_Nha vẫn chơi ép sân , liên_t